In [2]:
# Utils
import os, sys, re, time, gc, types, string, unicodedata, unidecode, string, warnings, inspect

#import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import time
import pickle
import re, sys, unidecode

from sklearn.metrics import accuracy_score
from IPython.display import display, Markdown, Latex

import shutil
import os

# Representation
import nltk
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD


import wordbatch
from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL

#plt.style.use('fivethirtyeight')
from nltk.stem.snowball import SnowballStemmer

#Carga stop word
nltk.download('stopwords')
spanish_stopwords = nltk.corpus.stopwords.words('spanish')

# punkt:  módulo contiene modelos para la tokenización de textos
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Parameters

In [3]:
root_path = "../"
path_data_clean = root_path + "data/clean/"
path_model = root_path + 'models/'
features_path = root_path + 'data/features/'
model_name = "tfidf10000_svd1000"
path_model += model_name

delete_old_model = True
if delete_old_model:
    try:
        os.system("rm -rf "+path_model)
        os.system("mkdir "+path_model)
    except:
        print("error")

# Load Data

falta agregar data v2 y beatiful soup para limpiar html en data 2

In [4]:
dfNoticias = pd.read_pickle(path_data_clean + "/dfNoticiasCleanV2.p")


In [5]:
dfNoticias.head()

,Cuerpo,Fecha,Hora,ID,Resumen,Seccion_1,Seccion_2,Seccion_3,Subtema_1,Subtema_2,Subtema_3,Tema_1,Tema_2,Tema_3,Titular,Type
0,"\nLa presidenta argentina, Cristina Fernández,...",2013-11-20,21:37,20131120214139,\nEste fue el primer acto público de la mandat...,mundo,None,None,None,None,None,argentina,None,None,\nPresidenta argentina tomó juramento a tres n...,fid_noticia
1,\nPor problemas de agenda de la jueza del Trib...,2013-11-20,17:09,20131120164254,\nAudiencia se realizará el próximo lunes 2 de...,pais,None,None,None,None,None,region de la araucania,None,None,\nNuevamente se postergó preparación del juici...,fid_noticia
2,"\nEl presidente uruguayo, José Mujica, sufre u...",2013-11-20,12:45,20131120124736,"\nPresidente uruguayo está afectado por un ""es...",mundo,None,None,None,None,None,uruguay,None,None,\nResfrío que afecta a José Mujica obligó a su...,fid_noticia
4,\nEl Reino Unido considera que el Ejecutivo es...,2013-11-20,11:43,20131120114123,\nGobierno hispano ha ejercido medidas de pres...,mundo,None,None,None,None,None,europa,None,None,\nReino Unido arremete contra España ante incu...,fid_noticia
5,\nFuncionarios municipales de Santiago y otras...,2013-11-20,12:44,20131120124031,\nUfemuch desconoce acuerdo con el Gobierno pa...,pais,None,None,gremios,None,None,trabajo,None,None,\nFuncionarios municipales de Santiago siguen ...,fid_noticia


# pre procesing



In [6]:
_map = {}
_map["Cuerpo"] = []
_map["Seccion"] = []
_map["Tema"] = []
_map["Subtema"] = []

count = 0
for index, row in dfNoticias.iterrows():    
    _map["Cuerpo"].append(row["Cuerpo"]) 
    _map["Seccion"].append(row["Seccion_1"])
    _map["Tema"].append(row["Tema_1"])
    _map["Subtema"].append(row["Subtema_1"])

df = pd.DataFrame(_map)
# Elimino clase corporativo, muy pocos ejemplos
df = df[df.Seccion != "corporativo"]

    
X_untransformed = df['Cuerpo'].reset_index(drop=True)
y1 = df['Seccion'].reset_index(drop=True)
y2 = df['Tema'].reset_index(drop=True)
y3 = df['Subtema'].reset_index(drop=True)


# Feature Extraccion

### Data representation
* Normalisacion
* TFID calculation

In [7]:

def normalize_text(text):
    """ Funcion de normalizacion """    
    # split into words
    tokens = nltk.tokenize.word_tokenize(text,language='spanish', preserve_line=False)
    # convert to lower case
    tokens = [w.lower() for w in tokens]    
    
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are n<<<<<<<<<<<<<<<<<<<<<
    words = [word for word in stripped if word.isalpha()]
    
    # stop word and remove accent
    def strip_accents(s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    stop_words = set(spanish_stopwords)
    words = [strip_accents(w) for w in words if not w in stop_words]
    return u" ".join(words)
    
#     stemmer = SnowballStemmer("spanish")
#     out = ""
#     for word in words:
#         out += stemmer.stem(word)+" "    
#     return out



# spanish_stopwords = stopwords.words('spanish')
# def normalize_text(text):
#     return u" ".join([x for x in [y for y in text.lower().strip().split(" ")] 
#                       if len(x) > 1 and x not in spanish_stopwords])

In [9]:

start = time.time()
print("start")


X_untransformed = X_untransformed
n_docs = X_untransformed.shape[0]
n_cpu = 40
#n_cpu = 6

batch_size = int(n_docs/n_cpu)

#'log', "idf":50.0

wb = wordbatch.WordBatch(normalize_text, 
                         extractor=(WordBag, {"hash_ngrams": 1, "hash_ngrams_weights": [1.0, 1.0],
                                              "hash_size": 2**28, "norm": "l2", "tf": 1.0,
                                              "idf": 1.0}), procs=n_cpu, n_words=10000, minibatch_size=batch_size)
                                              #"idf": 1.0}), procs=n_cpu, n_words=1000, minibatch_size=batch_size)



# wb = wordbatch.WordBatch(normalize_text, 
#                          extractor=(WordBag, {"hash_ngrams": 2, "hash_ngrams_weights": [1.0, 1.0],
#                                               "hash_size": 2**28, "norm": "l2", "tf": 1.0,
#                                               "idf": 1.0}), procs=n_cpu, n_words=10000, minibatch_size=batch_size)

# add method="serial" to extractor for debug normalize function

wb.dictionary_freeze = True
X_transformed_zero = wb.fit_transform(list(X_untransformed),reset= False)

non_zero_index_feat = np.array(np.clip(X_transformed_zero.getnnz(axis=0) - 1, 0, 1), dtype = bool)
X_transformed = X_transformed_zero[:, non_zero_index_feat]

print("TFIDF end time :" + str(time.time() - start) )

X = X_transformed

#X = X.todense()
print('Number of features: {}'.format(X.shape[1]))



start
Normalize text
Parallelization fail. Method: multiprocessing Task: <function batch_normalize_texts at 0x7fbcd1df3510>
Retrying, attempt: 1 timeout limit: 1200 seconds
Parallelization fail. Method: multiprocessing Task: <function batch_normalize_texts at 0x7fbcd1df3510>
Retrying, attempt: 2 timeout limit: 2400 seconds
Parallelization fail. Method: multiprocessing Task: <function batch_normalize_texts at 0x7fbcd1df3510>
Retrying, attempt: 3 timeout limit: 4800 seconds
Parallelization fail. Method: multiprocessing Task: <function batch_normalize_texts at 0x7fbcd1df3510>
Retrying, attempt: 4 timeout limit: 9600 seconds
Parallelization fail. Method: multiprocessing Task: <function batch_normalize_texts at 0x7fbcd1df3510>


TypeError: 'NoneType' object is not iterable

# SVD


Dim redution, libreria con implementacion multicore


In [ ]:
svd_enable = True
new_dim = 1000

start = time.time()

if svd_enable :
    start = time.time()
    svdT = TruncatedSVD(n_components = new_dim)
    X = svdT.fit_transform(X)
    print("end time :" + str(time.time() - start) )


## Save data

### save features

In [ ]:
save_features = False 
split_data = False
file_name = "tfid_hash28_n10000_svd1000_sin_stemmig.p"
train_fraction = 0.8

if split_data and save_features :
    y1 = y1.values    

    np.random.seed(42)
    train_indices = np.random.choice(X.shape[0], round(train_fraction*X.shape[0]), replace=False)
    test_indices = np.array(list(set(range(X.shape[0])) - set(train_indices)))

    X_train = X[train_indices]
    y_train = y1[train_indices]
    X_test = X[test_indices]
    y_test = y1[test_indices]
    features_path_ = features_path + "dataTrainTest_" + file_name
    pickle.dump( (X_train,y_train,X_test,y_test) , open( features_path_, "wb" ) )
elif save_features:
    features_path_ = features_path + "dataV2_" + file_name
    pickle.dump( (X,y1,y2,y3) , open( features_path_, "wb" ) )
    

### save models
* wordabatch calculator
* svd transform

In [ ]:
save_models = False
if save_models:
    file_name = "tfid_hash28_n10000_svd1000_sin_stemmig.p"
    features_path_ = features_path + "calcFeat_dataV2_" + file_name
    pickle.dump( (wb,svdT,non_zero_index_feat), open( features_path_, "wb" ) )